In [1]:
import random
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

In [4]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.6 MB/s eta 0:00:00


In [11]:
# df = pd.read_csv("imdb_dataset.csv")
from datasets import load_dataset
dataset = load_dataset("imdb")
df = pd.concat([pd.DataFrame(dataset["train"]),pd.DataFrame(dataset["test"])],axis=0)
print (df.shape)
df.rename(columns={'text':})

(50000, 2)


In [12]:
df.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [13]:
print (df.isnull().sum())

text     0
label    0
dtype: int64


In [14]:
reviews = list(df["text"])
sentiments = list(df["label"])
print (set(sentiments))

{0, 1}


In [15]:
balanced_reviews = reviews
balanced_sentiments = sentiments
print (len(balanced_reviews))
print (len(balanced_sentiments))
print (sum(balanced_sentiments))

50000
50000
25000


In [16]:
# positive_reviews = [r for r, s in zip(reviews, sentiments) if s == "positive"]
# negative_reviews = [r for r, s in zip(reviews, sentiments) if s == "negative"]

# pos_sample_size = int(0.60 * len(positive_reviews))
# neg_sample_size = int(0.60 * len(negative_reviews))

# sampled_positive = random.sample(positive_reviews, pos_sample_size)
# sampled_negative = random.sample(negative_reviews, neg_sample_size)

# balanced_reviews = sampled_positive + sampled_negative
# balanced_sentiments = ["positive"] * pos_sample_size + ["negative"] * neg_sample_size

# combined = list(zip(balanced_reviews, balanced_sentiments))
# random.shuffle(combined)
# balanced_reviews, balanced_sentiments = zip(*combined)

# balanced_reviews = list(balanced_reviews)
# balanced_sentiments = list(balanced_sentiments)

# print (len(balanced_reviews))
# print (len(balanced_sentiments))

In [17]:
X_train, X_temp, y_train, y_temp = train_test_split(balanced_reviews, balanced_sentiments, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print (len(X_train), len(y_train))
print (len(X_val), len(y_val))
print (len(X_test), len(y_test))

30000 30000
10000 10000
10000 10000


In [18]:
vectorizer = TfidfVectorizer(stop_words="english")
X_train_tfidf = vectorizer.fit_transform(X_train)
print (len(vectorizer.vocabulary_))

82357


In [19]:
vectorizer = TfidfVectorizer(stop_words="english", max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
print (len(vectorizer.vocabulary_))

10000


In [20]:
vectorizer = TfidfVectorizer(stop_words="english", max_df=0.95, min_df=0.05)
X_train_tfidf = vectorizer.fit_transform(X_train)
print (len(vectorizer.vocabulary_))

258


In [21]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,3), max_df=0.95, min_df=0.025)
X_train_tfidf = vectorizer.fit_transform(X_train, )
print (len(vectorizer.vocabulary_))

636


In [22]:
# vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
# X_train_tfidf = vectorizer.fit_transform(X_train)
# print (len(vectorizer.vocabulary_))

In [23]:
#vectorizer.vocabulary_

In [24]:
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

In [25]:
X_val_tfidf

<10000x636 sparse matrix of type '<class 'numpy.float64'>'
	with 404546 stored elements in Compressed Sparse Row format>

In [26]:
#X_val_tfidf[9999].data

In [27]:
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [28]:
y_train_pred = model.predict(X_train_tfidf)
val_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", val_accuracy)

Train Accuracy: 0.8567666666666667


In [29]:
y_val_pred = model.predict(X_val_tfidf)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: 0.849


In [30]:
y_test_pred = model.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.843


In [31]:
model = RandomForestClassifier(n_estimators=30)
model.fit(X_train_tfidf, y_train)

RandomForestClassifier(n_estimators=30)

In [32]:
y_train_pred = model.predict(X_train_tfidf)
val_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", val_accuracy)
y_val_pred = model.predict(X_val_tfidf)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
y_test_pred = model.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

Train Accuracy: 0.9996666666666667
Validation Accuracy: 0.8046
Test Accuracy: 0.8039


In [33]:
model = RandomForestClassifier(n_estimators=100, max_depth=20)
model.fit(X_train_tfidf, y_train)

RandomForestClassifier(max_depth=20)

In [34]:
y_train_pred = model.predict(X_train_tfidf)
val_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", val_accuracy)
y_val_pred = model.predict(X_val_tfidf)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
y_test_pred = model.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

Train Accuracy: 0.9032666666666667
Validation Accuracy: 0.7984
Test Accuracy: 0.7991


In [35]:
model = KNeighborsClassifier(n_neighbors=7)
model.fit(X_train_tfidf, y_train)

KNeighborsClassifier(n_neighbors=7)

In [36]:
y_train_pred = model.predict(X_train_tfidf)
val_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", val_accuracy)
y_val_pred = model.predict(X_val_tfidf)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
y_test_pred = model.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

Train Accuracy: 0.7952
Validation Accuracy: 0.7052
Test Accuracy: 0.699


In [37]:
model = XGBClassifier(n_estimators=50, max_depth=5)
model.fit(X_train_tfidf, y_train)

AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [38]:
y_train_pred = model.predict(X_train_tfidf)
val_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", val_accuracy)
y_val_pred = model.predict(X_val_tfidf)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
y_test_pred = model.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

Train Accuracy: 0.8686666666666667
Validation Accuracy: 0.8118
Test Accuracy: 0.8155


In [39]:
len(y_test_pred)

10000

In [40]:
def ensamble():
    model = LogisticRegression()
    model.fit(X_train_tfidf, y_train)
    y_pred1 = model.predict(X_test_tfidf)

    model = RandomForestClassifier(n_estimators=100, max_depth=20)
    model.fit(X_train_tfidf, y_train)
    y_pred2 = model.predict(X_test_tfidf)

    model = RandomForestClassifier(n_estimators=100, max_depth=20)
    model.fit(X_train_tfidf, y_train)
    y_pred3 = model.predict(X_test_tfidf)

    y_pred = []
    for i in range(len(y_pred1)):
        if y_pred1[i] + y_pred2[i] + y_pred3[i] >= 2:
            y_pred.append(1)
        else:
            y_pred.append(0)

    acc = accuracy_score(y_test, y_pred)
    return acc

In [ ]:
acc = ensamble()
print ("Enemble acc:", acc)